In [86]:
import os
import pandas as pd
import numpy as np
from collections import defaultdict

First, it is necessary to standardize the data in the HYPERAKTIVE dataset and then separate the data into two folders: condition (for patients with ADHD) and control (for healthy individuals). The goal is to adjust the data from this dataset to a universally accepted format compatible with the code written for handling it.

In [91]:
def create_condition_and_control(folder_path):
    # Load patient data
    patients_info_df = pd.read_csv('./data/hyperaktiv/patient_info.csv', sep = ';')
    patients_info_df = patients_info_df[['ID', 'ADHD']]
    condition_id = 1
    control_id = 1
    
    # Iterate through files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            patient_id = filename.split('_')[2].split('.')[0]  # Extract patient ID
            patient_overall_info_df = patients_info_df.loc[patients_info_df['ID'] == int(patient_id)]
            
            # Check if the patient has ADHD
            if not patient_overall_info_df.empty:
                diagnosed_adhd = int(patient_overall_info_df['ADHD'].iloc[0])  # Get the ADHD status
                
                file_path = os.path.join(folder_path, filename)
                patient_info_df = pd.read_csv(file_path, sep = ';')
                patient_info_df = patient_info_df.rename(columns = {'TIMESTAMP': 'timestamp', 'ACTIVITY': 'activity'})
                patient_info_df['timestamp'] = pd.to_datetime(patient_info_df['timestamp'])
                patient_info_df['date'] = patient_info_df['timestamp'].dt.date
                patient_info_df = patient_info_df[['timestamp', 'date', 'activity']]
                
                # Save the file to the appropriate folder
                if diagnosed_adhd == 1:  # Patient has ADHD
                    new_file_path = os.path.join('./data/hyperaktiv/condition/', f'condition_{condition_id}.csv')
                    patient_info_df.to_csv(new_file_path, index = False, sep = ',')
                    condition_id += 1
                elif diagnosed_adhd == 0:  # Patient does not have ADHD
                    new_file_path = os.path.join('./data/hyperaktiv/control/', f'control_{control_id}.csv')
                    patient_info_df.to_csv(new_file_path, index = False, sep = ',')
                    control_id += 1
                
                print(f'Saved: {new_file_path}')
            else:
                print(f'No data found for patient {patient_id}. Skipping...')

This code can be run once. It has already be run.

In [92]:
folder_path = './data/hyperaktiv/activity_data'

create_condition_and_control(folder_path)

Saved: ./data/hyperaktiv/condition/condition_1.csv
Saved: ./data/hyperaktiv/condition/condition_2.csv
Saved: ./data/hyperaktiv/condition/condition_3.csv
Saved: ./data/hyperaktiv/condition/condition_4.csv
Saved: ./data/hyperaktiv/control/control_1.csv
Saved: ./data/hyperaktiv/condition/condition_5.csv
Saved: ./data/hyperaktiv/control/control_2.csv
Saved: ./data/hyperaktiv/control/control_3.csv
Saved: ./data/hyperaktiv/control/control_4.csv
Saved: ./data/hyperaktiv/control/control_5.csv
Saved: ./data/hyperaktiv/control/control_6.csv
Saved: ./data/hyperaktiv/condition/condition_6.csv
Saved: ./data/hyperaktiv/condition/condition_7.csv
Saved: ./data/hyperaktiv/condition/condition_8.csv
Saved: ./data/hyperaktiv/condition/condition_9.csv
Saved: ./data/hyperaktiv/control/control_7.csv
Saved: ./data/hyperaktiv/control/control_8.csv
Saved: ./data/hyperaktiv/condition/condition_10.csv
Saved: ./data/hyperaktiv/condition/condition_11.csv
Saved: ./data/hyperaktiv/condition/condition_12.csv
Saved: ./

# Day

In [100]:
base_dir = 'data/hyperaktiv'
output_dir = 'data_day/hyperaktiv'
excluded_file_names = ['patient_info']
excluded_folder_names = ['activity_data']
os.makedirs(output_dir, exist_ok = True)

def split_time_series(file_path, start_hour=8, end_hour=21):
    df = pd.read_csv(file_path)
    
    if 'timestamp' not in df.columns:
        raise ValueError("The CSV file must have a 'timestamp' column.")
    
    # Ensure timestamp is in datetime format
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date  # Extract the date part of the timestamp
    
    # Sort by timestamp to ensure the correct order
    df = df.sort_values('timestamp')
    
    # Prepare the list for segments
    segments = []
    
    # Iterate through each unique date in the dataframe
    for unique_date in df['date'].unique():
        # Filter data for each date
        day_data = df[df['date'] == unique_date]
        
        # Filter for the time range (from start_hour to end_hour)
        day_data_filtered = day_data[
            (day_data['timestamp'].dt.hour >= start_hour) & 
            (day_data['timestamp'].dt.hour < end_hour)
        ]
        
        # If there's data within the specified time range, append to segments
        if not day_data_filtered.empty:
            segments.append(day_data_filtered)
    
    return segments

# Process all class folders
for class_folder in os.listdir(base_dir):
    if class_folder not in excluded_folder_names:
        class_path = os.path.join(base_dir, class_folder)
        if not os.path.isdir(class_path):
            continue  # Skip non-folder entries
    
        print(f"Processing class: {class_folder}")
        output_class_dir = os.path.join(output_dir, class_folder)
        os.makedirs(output_class_dir, exist_ok=True)
    
        for file_name in os.listdir(class_path):
            if file_name.endswith('.csv') and file_name not in excluded_file_names:
                file_path = os.path.join(class_path, file_name)
                segments = split_time_series(file_path)
            
                # Save each segment as a new CSV file
                for idx, segment in enumerate(segments):
                    segment_file = os.path.join(output_class_dir, f"{file_name[:-4]}_segment_{idx}.csv")
                    segment.to_csv(segment_file, index=False)
                    print(f"Saved: {segment_file}")

Processing class: condition
Saved: data_day/hyperaktiv\condition\condition_1_segment_0.csv
Saved: data_day/hyperaktiv\condition\condition_1_segment_1.csv
Saved: data_day/hyperaktiv\condition\condition_1_segment_2.csv
Saved: data_day/hyperaktiv\condition\condition_1_segment_3.csv
Saved: data_day/hyperaktiv\condition\condition_1_segment_4.csv
Saved: data_day/hyperaktiv\condition\condition_1_segment_5.csv
Saved: data_day/hyperaktiv\condition\condition_1_segment_6.csv
Saved: data_day/hyperaktiv\condition\condition_10_segment_0.csv
Saved: data_day/hyperaktiv\condition\condition_10_segment_1.csv
Saved: data_day/hyperaktiv\condition\condition_10_segment_2.csv
Saved: data_day/hyperaktiv\condition\condition_10_segment_3.csv
Saved: data_day/hyperaktiv\condition\condition_10_segment_4.csv
Saved: data_day/hyperaktiv\condition\condition_10_segment_5.csv
Saved: data_day/hyperaktiv\condition\condition_10_segment_6.csv
Saved: data_day/hyperaktiv\condition\condition_10_segment_7.csv
Saved: data_day/hyp

In [104]:
def count_rows_in_csv_files(folder_path):
    # Tworzymy defaultdict, aby automatycznie tworzył listy dla każdej liczby wierszy
    row_counts_dict = defaultdict(int)
    
    # Iterujemy przez pliki w folderze
    for filename in os.listdir(folder_path):
        # Sprawdzamy, czy plik ma rozszerzenie .csv
        if filename.endswith('.csv') and file_name not in excluded_file_names:
            file_path = os.path.join(folder_path, filename)
            
            # Wczytujemy plik CSV
            try:
                df = pd.read_csv(file_path)
                row_count = len(df)
                # Zwiększamy licznik dla tej liczby wierszy
                row_counts_dict[row_count] += 1
            except Exception as e:
                # Obsługuje ewentualne błędy podczas otwierania pliku
                print(f"Error reading {filename}: {e}")
    
    return row_counts_dict

# Ścieżka do folderu z plikami CSV
folder_path = './data_day/hyperaktiv/control' # tu wpisuje sie depresjo, psykose lub hyperaktiv a potem condition a nastepnie control 


# Wywołanie funkcji i wyświetlenie wyników
row_counts_grouped = count_rows_in_csv_files(folder_path)

# Wypisanie wyników w wymaganej formie
for row_count, file_count in row_counts_grouped.items():
    print(f"Długość {row_count}: {file_count} plików")

Długość 615: 2 plików
Długość 780: 254 plików
Długość 716: 1 plików
Długość 540: 1 plików
Długość 371: 1 plików
Długość 555: 1 plików
Długość 599: 1 plików
Długość 274: 1 plików
Długość 468: 1 plików
Długość 660: 4 plików
Długość 595: 1 plików
Długość 613: 1 plików
Długość 369: 1 plików
Długość 388: 1 plików
Długość 746: 1 plików
Długość 405: 1 plików
Długość 277: 1 plików
Długość 360: 3 plików
Długość 685: 1 plików
Długość 390: 2 plików
Długość 345: 1 plików
Długość 608: 1 plików
Długość 79: 1 plików
Długość 545: 1 plików
Długość 600: 2 plików
Długość 619: 1 plików
Długość 300: 2 plików
Długość 107: 1 plików
Długość 668: 1 plików
Długość 125: 1 plików
Długość 543: 1 plików
Długość 669: 1 plików
Długość 299: 1 plików
Długość 481: 1 plików
Długość 415: 1 plików
Długość 321: 1 plików
Długość 532: 2 plików
Długość 265: 1 plików
Długość 480: 1 plików
Długość 398: 1 plików
Długość 633: 1 plików
Długość 101: 1 plików
Długość 225: 1 plików
Długość 450: 1 plików
Długość 407: 2 plików
Długość 6

In [105]:
len(row_counts_grouped)

58

For the daytime period, the expected number of rows is 780. This is calculated as follows: the duration of the day is from 8:00 to 21:00, which equals 13 hours. Since there are 60 minutes in an hour, the total is 13 * 60 = 780 minutes.

In [106]:
# Ścieżka do folderu zawierającego pliki CSV
folder_path = './data_day/hyperaktiv/control'  

# Iterujemy przez wszystkie pliki w folderze
for file_name in os.listdir(folder_path):
    # Sprawdzamy, czy plik ma rozszerzenie .csv
    if file_name.endswith('.csv') and file_name not in excluded_file_names:
        file_path = os.path.join(folder_path, file_name)
        
        try:
            # Wczytujemy plik CSV do pandas DataFrame
            df = pd.read_csv(file_path)
            
            # Sprawdzamy liczbę wierszy w pliku
            if len(df) != 780: # ustawienie wartosci, jesli sie nie zgadza usuwane sa bledne pliki
                print(f"Usuwam plik: {file_name} (zawiera {len(df)} linijek)")
                os.remove(file_path)  # Usuwamy plik
        except Exception as e:
            print(f"Błąd przy przetwarzaniu pliku {file_name}: {e}")

Usuwam plik: control_10_segment_0.csv (zawiera 615 linijek)
Usuwam plik: control_10_segment_7.csv (zawiera 716 linijek)
Usuwam plik: control_11_segment_0.csv (zawiera 540 linijek)
Usuwam plik: control_11_segment_6.csv (zawiera 371 linijek)
Usuwam plik: control_12_segment_0.csv (zawiera 555 linijek)
Usuwam plik: control_13_segment_0.csv (zawiera 599 linijek)
Usuwam plik: control_13_segment_7.csv (zawiera 274 linijek)
Usuwam plik: control_14_segment_0.csv (zawiera 468 linijek)
Usuwam plik: control_15_segment_0.csv (zawiera 660 linijek)
Usuwam plik: control_15_segment_7.csv (zawiera 595 linijek)
Usuwam plik: control_16_segment_0.csv (zawiera 613 linijek)
Usuwam plik: control_16_segment_7.csv (zawiera 369 linijek)
Usuwam plik: control_17_segment_0.csv (zawiera 388 linijek)
Usuwam plik: control_17_segment_7.csv (zawiera 746 linijek)
Usuwam plik: control_18_segment_0.csv (zawiera 405 linijek)
Usuwam plik: control_18_segment_7.csv (zawiera 277 linijek)
Usuwam plik: control_19_segment_0.csv (z

# Night

In [107]:
base_dir = 'data/hyperaktiv'
output_dir = 'data_night/hyperaktiv'
excluded_file_names = ['patient_info']
excluded_folder_names = ['activity_data']
os.makedirs(output_dir, exist_ok = True)

def split_time_series(file_path, start_hour=21, end_hour=8):
    df = pd.read_csv(file_path)
    
    if 'timestamp' not in df.columns:
        raise ValueError("The CSV file must have a 'timestamp' column.")
    
    # Ensure timestamp is in datetime format
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date  # Extract the date part of the timestamp
    
    # Sort by timestamp to ensure the correct order
    df = df.sort_values('timestamp')
    
    # Prepare the list for segments
    segments = []
    
    # Iterate through each unique date in the dataframe
    for unique_date in df['date'].unique():
        # Filter data for the current date
        day_data = df[df['date'] == unique_date]
        
        # Handle times before midnight (21:00 to 23:59)
        night_data_part1 = day_data[day_data['timestamp'].dt.hour >= start_hour]
        
        # Handle times after midnight (00:00 to 08:00)
        next_date = unique_date + pd.Timedelta(days=1)  # Get the next day's date
        night_data_part2 = df[
            (df['date'] == next_date) & (df['timestamp'].dt.hour < end_hour)
        ]
        
        # Combine both parts of the night data
        night_data = pd.concat([night_data_part1, night_data_part2])
        
        # If there's data within the specified time range, append to segments
        if not night_data.empty:
            segments.append(night_data)
    
    return segments

# Process all class folders
for class_folder in os.listdir(base_dir):
    if class_folder not in excluded_folder_names:
        class_path = os.path.join(base_dir, class_folder)
        if not os.path.isdir(class_path):
            continue  # Skip non-folder entries
    
        print(f"Processing class: {class_folder}")
        output_class_dir = os.path.join(output_dir, class_folder)
        os.makedirs(output_class_dir, exist_ok=True)
    
        for file_name in os.listdir(class_path):
            if file_name.endswith('.csv') and file_name not in excluded_file_names:
                file_path = os.path.join(class_path, file_name)
                segments = split_time_series(file_path)
            
                # Save each segment as a new CSV file
                for idx, segment in enumerate(segments):
                    segment_file = os.path.join(output_class_dir, f"{file_name[:-4]}_segment_{idx}.csv")
                    segment.to_csv(segment_file, index=False)
                    print(f"Saved: {segment_file}")

Processing class: condition
Saved: data_night/hyperaktiv\condition\condition_1_segment_0.csv
Saved: data_night/hyperaktiv\condition\condition_1_segment_1.csv
Saved: data_night/hyperaktiv\condition\condition_1_segment_2.csv
Saved: data_night/hyperaktiv\condition\condition_1_segment_3.csv
Saved: data_night/hyperaktiv\condition\condition_1_segment_4.csv
Saved: data_night/hyperaktiv\condition\condition_1_segment_5.csv
Saved: data_night/hyperaktiv\condition\condition_10_segment_0.csv
Saved: data_night/hyperaktiv\condition\condition_10_segment_1.csv
Saved: data_night/hyperaktiv\condition\condition_10_segment_2.csv
Saved: data_night/hyperaktiv\condition\condition_10_segment_3.csv
Saved: data_night/hyperaktiv\condition\condition_10_segment_4.csv
Saved: data_night/hyperaktiv\condition\condition_10_segment_5.csv
Saved: data_night/hyperaktiv\condition\condition_10_segment_6.csv
Saved: data_night/hyperaktiv\condition\condition_11_segment_0.csv
Saved: data_night/hyperaktiv\condition\condition_11_se

In [111]:
def count_rows_in_csv_files(folder_path):
    # Tworzymy defaultdict, aby automatycznie tworzył listy dla każdej liczby wierszy
    row_counts_dict = defaultdict(int)
    
    # Iterujemy przez pliki w folderze
    for filename in os.listdir(folder_path):
        # Sprawdzamy, czy plik ma rozszerzenie .csv
        if filename.endswith('.csv') and file_name not in excluded_file_names:
            file_path = os.path.join(folder_path, filename)
            
            # Wczytujemy plik CSV
            try:
                df = pd.read_csv(file_path)
                row_count = len(df)
                # Zwiększamy licznik dla tej liczby wierszy
                row_counts_dict[row_count] += 1
            except Exception as e:
                # Obsługuje ewentualne błędy podczas otwierania pliku
                print(f"Error reading {filename}: {e}")
    
    return row_counts_dict

# Ścieżka do folderu z plikami CSV
folder_path = './data_night/hyperaktiv/control' # tu wpisuje sie depresjo, psykose lub hyperaktiv a potem condition a nastepnie control 


# Wywołanie funkcji i wyświetlenie wyników
row_counts_grouped = count_rows_in_csv_files(folder_path)

# Wypisanie wyników w wymaganej formie
for row_count, file_count in row_counts_grouped.items():
    print(f"Długość {row_count}: {file_count} plików")

Długość 660: 282 plików
Długość 181: 1 plików
Długość 394: 1 plików
Długość 405: 1 plików
Długość 371: 1 plików
Długość 48: 1 plików
Długość 110: 1 plików
Długość 131: 1 plików
Długość 314: 1 plików
Długość 193: 1 plików
Długość 96: 1 plików
Długość 415: 1 plików
Długość 112: 1 plików


In [112]:
len(row_counts_grouped)

13

For the nighttime period, the expected number of rows is 780. This is calculated as follows: the duration of the night is from 21:00 to 08:00, which equals 11 hours. Since there are 60 minutes in an hour, the total is 11 * 60 = 660 minutes.

In [113]:
# Ścieżka do folderu zawierającego pliki CSV
folder_path = './data_night/hyperaktiv/control'  

# Iterujemy przez wszystkie pliki w folderze
for file_name in os.listdir(folder_path):
    # Sprawdzamy, czy plik ma rozszerzenie .csv
    if file_name.endswith('.csv') and file_name not in excluded_file_names:
        file_path = os.path.join(folder_path, file_name)
        
        try:
            # Wczytujemy plik CSV do pandas DataFrame
            df = pd.read_csv(file_path)
            
            # Sprawdzamy liczbę wierszy w pliku
            if len(df) != 660: # ustawienie wartosci poniżej ktorej usuwa sie bledne pliki
                print(f"Usuwam plik: {file_name} (zawiera {len(df)} linijek)")
                os.remove(file_path)  # Usuwamy plik
        except Exception as e:
            print(f"Błąd przy przetwarzaniu pliku {file_name}: {e}")

Usuwam plik: control_12_segment_8.csv (zawiera 181 linijek)
Usuwam plik: control_14_segment_9.csv (zawiera 394 linijek)
Usuwam plik: control_21_segment_6.csv (zawiera 405 linijek)
Usuwam plik: control_22_segment_6.csv (zawiera 371 linijek)
Usuwam plik: control_27_segment_7.csv (zawiera 48 linijek)
Usuwam plik: control_29_segment_7.csv (zawiera 110 linijek)
Usuwam plik: control_34_segment_7.csv (zawiera 131 linijek)
Usuwam plik: control_36_segment_6.csv (zawiera 314 linijek)
Usuwam plik: control_37_segment_6.csv (zawiera 193 linijek)
Usuwam plik: control_40_segment_8.csv (zawiera 96 linijek)
Usuwam plik: control_4_segment_6.csv (zawiera 415 linijek)
Usuwam plik: control_6_segment_6.csv (zawiera 112 linijek)


# Full 24h

In [93]:
base_dir = 'data/hyperaktiv'
output_dir = 'data_24h/hyperaktiv'
excluded_file_names = ['patient_info']
excluded_folder_names = ['activity_data']
os.makedirs(output_dir, exist_ok = True)

def split_time_series(file_path, segment_duration = 24):
    """
    Splits a time series CSV file into 24-hour segments.
    
    Parameters:
    - file_path: str, path to the CSV file
    - segment_duration: int, duration of each segment (in hours)
    
    Returns:
    - List of DataFrames, each representing a segment
    """
    df = pd.read_csv(file_path)
    if 'timestamp' not in df.columns:
        raise ValueError("The CSV file must have a 'timestamp' column.")
    
    # Ensure timestamp is in datetime format and sorted
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp')
    
    # Calculate 24-hour chunks (assuming the data is hourly sampled)
    start_time = df['timestamp'].min()
    end_time = df['timestamp'].max()
    total_hours = int((end_time - start_time).total_seconds() // 3600)
    
    segments = []
    for i in range(0, total_hours, segment_duration):
        segment = df[
            (df['timestamp'] >= start_time + pd.Timedelta(hours=i)) &
            (df['timestamp'] < start_time + pd.Timedelta(hours=i + segment_duration))
        ]
        if not segment.empty:
            segments.append(segment)
    
    return segments

# Process all class folders
for class_folder in os.listdir(base_dir):
    if class_folder not in excluded_folder_names:
        class_path = os.path.join(base_dir, class_folder)
        if not os.path.isdir(class_path):
            continue  # Skip non-folder entries
    
        print(f"Processing class: {class_folder}")
        output_class_dir = os.path.join(output_dir, class_folder)
        os.makedirs(output_class_dir, exist_ok=True)
    
        for file_name in os.listdir(class_path):
            if file_name.endswith('.csv') and file_name not in excluded_file_names:
                file_path = os.path.join(class_path, file_name)
                segments = split_time_series(file_path)
            
                # Save each segment as a new CSV file
                for idx, segment in enumerate(segments):
                    segment_file = os.path.join(output_class_dir, f"{file_name[:-4]}_segment_{idx}.csv")
                    segment.to_csv(segment_file, index=False)
                    print(f"Saved: {segment_file}")

Processing class: condition
Saved: data_24h/hyperaktiv\condition\condition_1_segment_0.csv
Saved: data_24h/hyperaktiv\condition\condition_1_segment_1.csv
Saved: data_24h/hyperaktiv\condition\condition_1_segment_2.csv
Saved: data_24h/hyperaktiv\condition\condition_1_segment_3.csv
Saved: data_24h/hyperaktiv\condition\condition_1_segment_4.csv
Saved: data_24h/hyperaktiv\condition\condition_1_segment_5.csv
Saved: data_24h/hyperaktiv\condition\condition_10_segment_0.csv
Saved: data_24h/hyperaktiv\condition\condition_10_segment_1.csv
Saved: data_24h/hyperaktiv\condition\condition_10_segment_2.csv
Saved: data_24h/hyperaktiv\condition\condition_10_segment_3.csv
Saved: data_24h/hyperaktiv\condition\condition_10_segment_4.csv
Saved: data_24h/hyperaktiv\condition\condition_10_segment_5.csv
Saved: data_24h/hyperaktiv\condition\condition_10_segment_6.csv
Saved: data_24h/hyperaktiv\condition\condition_10_segment_7.csv
Saved: data_24h/hyperaktiv\condition\condition_11_segment_0.csv
Saved: data_24h/hy

In [97]:
def count_rows_in_csv_files(folder_path):
    # Tworzymy defaultdict, aby automatycznie tworzył listy dla każdej liczby wierszy
    row_counts_dict = defaultdict(int)
    
    # Iterujemy przez pliki w folderze
    for filename in os.listdir(folder_path):
        # Sprawdzamy, czy plik ma rozszerzenie .csv
        if filename.endswith('.csv') and file_name not in excluded_file_names:
            file_path = os.path.join(folder_path, filename)
            
            # Wczytujemy plik CSV
            try:
                df = pd.read_csv(file_path)
                row_count = len(df)
                # Zwiększamy licznik dla tej liczby wierszy
                row_counts_dict[row_count] += 1
            except Exception as e:
                # Obsługuje ewentualne błędy podczas otwierania pliku
                print(f"Error reading {filename}: {e}")
    
    return row_counts_dict

# Ścieżka do folderu z plikami CSV
folder_path = './data_24h/hyperaktiv/control' # tu wpisuje sie depresjo, psykose lub hyperaktiv a potem condition a nastepnie control 


# Wywołanie funkcji i wyświetlenie wyników
row_counts_grouped = count_rows_in_csv_files(folder_path)

# Wypisanie wyników w wymaganej formie
for row_count, file_count in row_counts_grouped.items():
    print(f"Długość {row_count}: {file_count} plików")

Długość 1440: 271 plików
Długość 551: 1 plików
Długość 131: 1 plików
Długość 736: 1 plików
Długość 93: 1 plików
Długość 862: 1 plików
Długość 475: 1 plików
Długość 202: 1 plików
Długość 354: 1 plików
Długość 1342: 1 plików
Długość 265: 1 plików
Długość 1395: 1 plików
Długość 1347: 1 plików
Długość 950: 1 plików
Długość 971: 1 plików
Długość 499: 1 plików
Długość 1067: 1 plików
Długość 123: 1 plików
Długość 717: 1 plików
Długość 725: 1 plików
Długość 1396: 1 plików
Długość 98: 1 plików
Długość 1394: 1 plików
Długość 105: 1 plików
Długość 581: 1 plików
Długość 1136: 1 plików
Długość 674: 1 plików
Długość 897: 1 plików
Długość 1099: 1 plików
Długość 1338: 1 plików
Długość 503: 1 plików
Długość 805: 1 plików
Długość 485: 1 plików
Długość 622: 1 plików
Długość 1359: 1 plików
Długość 156: 1 plików
Długość 1150: 1 plików


In [98]:
len(row_counts_grouped)

37

In [99]:
# Ścieżka do folderu zawierającego pliki CSV
folder_path = './data_24h/hyperaktiv/control'  

# Iterujemy przez wszystkie pliki w folderze
for file_name in os.listdir(folder_path):
    # Sprawdzamy, czy plik ma rozszerzenie .csv
    if file_name.endswith('.csv') and file_name not in excluded_file_names:
        file_path = os.path.join(folder_path, file_name)
        
        try:
            # Wczytujemy plik CSV do pandas DataFrame
            df = pd.read_csv(file_path)
            
            # Sprawdzamy liczbę wierszy w pliku
            if len(df) != 1440: # ustawienie wartosci poniżej ktorej usuwa sie bledne pliki
                print(f"Usuwam plik: {file_name} (zawiera {len(df)} linijek)")
                os.remove(file_path)  # Usuwamy plik
        except Exception as e:
            print(f"Błąd przy przetwarzaniu pliku {file_name}: {e}")

Usuwam plik: control_10_segment_7.csv (zawiera 551 linijek)
Usuwam plik: control_11_segment_6.csv (zawiera 131 linijek)
Usuwam plik: control_12_segment_8.csv (zawiera 736 linijek)
Usuwam plik: control_13_segment_7.csv (zawiera 93 linijek)
Usuwam plik: control_14_segment_9.csv (zawiera 862 linijek)
Usuwam plik: control_15_segment_7.csv (zawiera 475 linijek)
Usuwam plik: control_16_segment_7.csv (zawiera 202 linijek)
Usuwam plik: control_17_segment_7.csv (zawiera 354 linijek)
Usuwam plik: control_18_segment_6.csv (zawiera 1342 linijek)
Usuwam plik: control_19_segment_7.csv (zawiera 265 linijek)
Usuwam plik: control_1_segment_7.csv (zawiera 1395 linijek)
Usuwam plik: control_20_segment_6.csv (zawiera 1347 linijek)
Usuwam plik: control_21_segment_6.csv (zawiera 950 linijek)
Usuwam plik: control_22_segment_6.csv (zawiera 971 linijek)
Usuwam plik: control_23_segment_7.csv (zawiera 499 linijek)
Usuwam plik: control_24_segment_6.csv (zawiera 1067 linijek)
Usuwam plik: control_26_segment_7.csv 